In [1]:
# Import libraries and set desired options
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from scipy import sparse, stats
from scipy.linalg import svd
import umap
from sklearn import preprocessing
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import (KFold, StratifiedKFold, cross_val_score,
                                     cross_validate, train_test_split)
from tqdm import tqdm
from lightgbm import LGBMClassifier
from code.cross_validation import *
from code.read_data import *
from code.feature_engineering import *

In [2]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
sns.set()
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

# Read data sets

In [3]:
X1, X2, X3, Y, X1_test, X2_test, X3_test = read_data()
Y = Y.rename(columns={f'{i}': f'target_{i}' for i in range(1, 6)})

In [4]:
X_train = agg_and_merge(X1, X2, X3)
X_test = agg_and_merge(X1_test, X2_test, X3_test)
X_train, X_test = eng(X_train, X_test)
X_train, X_test = add_emedding_features(X_train, X_test, random_state=42)
X_train_norm, X_test_norm = normalize(X_train, X_test)

# Simple models

In [5]:
merged = pd.merge(X_train, Y, on='id')
logreg_merged = pd.merge(X_train_norm, Y, on='id')
targets = ['target_1', 'target_2', 'target_3', 'target_4', 'target_5']
assert len(merged) == len(logreg_merged)
final_models = {}
final_models_roc_auc = {}

## logregs

In [6]:
C_grids = {
    'target_1': np.linspace(0.01, 0.1, 10),
    'target_2': np.linspace(0.01, 0.1, 10),
    'target_3': np.linspace(0.001, 0.01, 10),
    'target_4': np.linspace(0.005, 0.1, 10),
    'target_5': np.linspace(0.01, 0.1, 10),
}

for target in targets:
    print(target)
    roc_auc_means = []
    for C in C_grids[target]:
        logreg = LogisticRegression(C=C, class_weight='balanced', random_state=42, n_jobs=4)
        roc_auc_means.append(CV_metrics(logreg, X_train_norm, Y[target].values))
    print(roc_auc_means)
    index = roc_auc_means.index(max(roc_auc_means))
    print(f'index: {index}')
    C = C_grids[target][index]
    final_models[f'{target}_logreg'] = LogisticRegression(C=C, class_weight='balanced', random_state=42, n_jobs=-1)
    final_models_roc_auc[f'{target}_logreg'] = max(roc_auc_means)

target_1
roc_auc  avg:  0.537   ['0.510', '0.533', '0.567']
roc_auc  avg:  0.540   ['0.511', '0.536', '0.573']
roc_auc  avg:  0.541   ['0.512', '0.536', '0.576']
roc_auc  avg:  0.542   ['0.513', '0.536', '0.577']
roc_auc  avg:  0.543   ['0.514', '0.536', '0.578']
roc_auc  avg:  0.543   ['0.514', '0.536', '0.578']
roc_auc  avg:  0.543   ['0.514', '0.536', '0.579']
roc_auc  avg:  0.542   ['0.514', '0.535', '0.578']
roc_auc  avg:  0.543   ['0.514', '0.536', '0.578']
roc_auc  avg:  0.542   ['0.513', '0.535', '0.578']
[0.5367297211189884, 0.5400817943431638, 0.5412440964198096, 0.5419933159076616, 0.5426625767203841, 0.5426147613295506, 0.5426697693160621, 0.5423300723971892, 0.5425376056091477, 0.5421760091744451]
index: 6
target_2
roc_auc  avg:  0.567   ['0.564', '0.577', '0.560']
roc_auc  avg:  0.569   ['0.566', '0.579', '0.562']
roc_auc  avg:  0.569   ['0.566', '0.579', '0.563']
roc_auc  avg:  0.570   ['0.567', '0.579', '0.564']
roc_auc  avg:  0.570   ['0.567', '0.579', '0.564']
roc_auc

## RF

In [7]:
C_grids = {
    'target_1': [{'n_estimators':400, 'max_depth':5}, {'n_estimators':500, 'max_depth':5}, {'n_estimators':600, 'max_depth':5}],
    'target_2': [{'n_estimators':600, 'max_depth':7}, {'n_estimators':700, 'max_depth':7}, {'n_estimators':800, 'max_depth':7}],
    'target_3': [{'n_estimators':500, 'max_depth':5}, {'n_estimators':600, 'max_depth':5}, {'n_estimators':700, 'max_depth':5}],
    'target_4': [{'n_estimators':500, 'max_depth':3}, {'n_estimators':600, 'max_depth':3}, {'n_estimators':700, 'max_depth':3}],
    'target_5': [{'n_estimators':550, 'max_depth':3}, {'n_estimators':575, 'max_depth':3}, {'n_estimators':600, 'max_depth':3}],
}

for target in targets:
    print(target)
    roc_auc_means = []
    for options in C_grids[target]:
        rf = RandomForestClassifier(**options, class_weight='balanced', random_state=42, n_jobs=4)
        roc_auc_means.append(CV_metrics(rf, X_train, Y[target].values))
    print(roc_auc_means)
    index = roc_auc_means.index(max(roc_auc_means))
    print(f'index: {index}')
    options = C_grids[target][index]
    final_models[f'{target}_rf'] = RandomForestClassifier(**options,
                                                          class_weight='balanced',
                                                          random_state=42,
                                                          n_jobs=-1)
    final_models_roc_auc[f'{target}_rf'] = max(roc_auc_means)

target_1
roc_auc  avg:  0.564   ['0.532', '0.576', '0.584']
roc_auc  avg:  0.562   ['0.529', '0.572', '0.584']
roc_auc  avg:  0.561   ['0.527', '0.572', '0.584']
[0.5638886282599243, 0.5618735701684966, 0.5608971099092774]
index: 0
target_2
roc_auc  avg:  0.582   ['0.590', '0.584', '0.571']
roc_auc  avg:  0.583   ['0.593', '0.585', '0.572']
roc_auc  avg:  0.583   ['0.592', '0.586', '0.571']
[0.5818256347541609, 0.5832763623752149, 0.5831736264253847]
index: 1
target_3
roc_auc  avg:  0.617   ['0.607', '0.617', '0.627']
roc_auc  avg:  0.618   ['0.606', '0.616', '0.630']
roc_auc  avg:  0.617   ['0.605', '0.616', '0.629']
[0.6170897112745704, 0.6177441638995389, 0.6168164985349227]
index: 1
target_4
roc_auc  avg:  0.585   ['0.597', '0.587', '0.570']
roc_auc  avg:  0.584   ['0.596', '0.585', '0.572']
roc_auc  avg:  0.585   ['0.597', '0.585', '0.572']
[0.5845396549218451, 0.5844992553063606, 0.5846862691919629]
index: 2
target_5
roc_auc  avg:  0.535   ['0.534', '0.527', '0.543']
roc_auc  avg

## GBM

In [8]:
C_grids = {
    'target_1': [{'n_estimators':200, 'num_leaves':3, 'reg_lambda':10.0}, {'n_estimators':300, 'num_leaves':3, 'reg_lambda':10.0}, {'n_estimators':400, 'num_leaves':3, 'reg_lambda':10.0}],
    'target_2': [{'n_estimators':400, 'num_leaves':7, 'reg_lambda':10.0}, {'n_estimators':500, 'num_leaves':7, 'reg_lambda':10.0}, {'n_estimators':600, 'num_leaves':7, 'reg_lambda':10.0}],
    'target_3': [{'n_estimators':150, 'num_leaves':3, 'reg_lambda':5.0}, {'n_estimators':150, 'num_leaves':5, 'reg_lambda':10.0}, {'n_estimators':150, 'num_leaves':7, 'reg_lambda':15.0}],
    'target_4': [{'n_estimators':100, 'num_leaves':3, 'reg_lambda':10.0}, {'n_estimators':150, 'num_leaves':3, 'reg_lambda':10.0}, {'n_estimators':200, 'num_leaves':3, 'reg_lambda':10.0}],
    'target_5': [{'n_estimators':100, 'num_leaves':3, 'reg_lambda':10.0}, {'n_estimators':200, 'num_leaves':3, 'reg_lambda':10.0}, {'n_estimators':300, 'num_leaves':3, 'reg_lambda':10.0}],
}

for target in targets:
    print(target)
    roc_auc_means = []
    for options in C_grids[target]:
        gbm = LGBMClassifier(**options, learning_rate=0.01, random_state=42, class_weight='balanced')
        roc_auc_means.append(CV_metrics(gbm, X_train, Y[target].values))
    print(roc_auc_means)
    index = roc_auc_means.index(max(roc_auc_means))
    print(f'index: {index}')
    options = C_grids[target][index]
    final_models[f'{target}_gbm'] = LGBMClassifier(**options, random_state=42, class_weight='balanced')
    final_models_roc_auc[f'{target}_gbm'] = max(roc_auc_means)

target_1
roc_auc  avg:  0.591   ['0.586', '0.591', '0.594']
roc_auc  avg:  0.589   ['0.583', '0.591', '0.594']
roc_auc  avg:  0.589   ['0.580', '0.592', '0.595']
[0.590557138613978, 0.5894142557819152, 0.5889836055598097]
index: 0
target_2
roc_auc  avg:  0.587   ['0.596', '0.587', '0.578']
roc_auc  avg:  0.588   ['0.596', '0.587', '0.581']
roc_auc  avg:  0.588   ['0.596', '0.587', '0.581']
[0.5868844683394169, 0.5879345425359006, 0.5879373403797268]
index: 2
target_3
roc_auc  avg:  0.616   ['0.600', '0.617', '0.629']
roc_auc  avg:  0.616   ['0.605', '0.618', '0.625']
roc_auc  avg:  0.617   ['0.607', '0.621', '0.624']
[0.6155342207717028, 0.6160537026887066, 0.6170272922996122]
index: 2
target_4
roc_auc  avg:  0.598   ['0.611', '0.600', '0.582']
roc_auc  avg:  0.595   ['0.612', '0.598', '0.574']
roc_auc  avg:  0.593   ['0.610', '0.602', '0.567']
[0.5975346921776973, 0.5949037841249799, 0.5927887581201986]
index: 0
target_5
roc_auc  avg:  0.538   ['0.551', '0.527', '0.537']
roc_auc  avg:

In [11]:
np.mean([v for _, v in final_models_roc_auc.items()])

0.5785226926706261

## Model

In [9]:
probas = []
X_train = merged.drop(columns=targets)
X_train_logreg = logreg_merged.drop(columns=targets)
for target in targets:
    y_train = merged[target].values
    # models
    # lightgbm
    y_proba = final_models[f'{target}_gbm'].fit(X_train, y_train).predict_proba(X_test)[:, 1] * 0.2
    # random forest
    y_proba += final_models[f'{target}_rf'].fit(X_train, y_train).predict_proba(X_test)[:, 1] * 0.3
    # logreg
    y_proba += final_models[f'{target}_logreg'].fit(X_train_norm, y_train).predict_proba(X_test_norm)[:, 1] * 0.5
    probas.append(y_proba)

In [10]:
tmp = pd.DataFrame(probas).T
baseline = pd.DataFrame(tmp.values, columns=['1', '2', '3', '4', '5'])
baseline['id'] = X_test['id']
baseline[['id', '1', '2', '3', '4', '5']].to_csv('baseline.csv', index=False)